In [74]:
import math 


def generate_lattice_matrix(q, r, k=12):
    # Compute the Euler's totient function of k
    phi_k = euler_phi(k)
    # Initialize the matrix of size phi_k x phi_k
    L = Matrix(ZZ, phi_k, phi_k)
    # Set the first row of the matrix
    L[0, 0] = r
    for j in range(1, phi_k):
        L[0, j] = 0
    # Fill the rest of the matrix
    for i in range(1, phi_k):
        # q^i and it should be negative
        L[i, 0] = -q^i
        # Identity matrix part shifted by 1 row
        for j in range(1, phi_k):
            if i == j:
                L[i, j] = 1
            else:
                L[i, j] = 0
    return L
    
def find_vectors_and_sort_by_norm(L, x):
    # Compute the LLL-reduced basis of the lattice and print its coefficients relative to the seed x. 
    L_reduced = L.LLL()
    vectors_and_norms = []
    for vec in L_reduced.rows():
        # Calculate the Euclidean norm of the vector
        norm = vec.norm()
        vectors_and_norms.append((vec, norm))
    vectors_and_norms.sort(key=lambda x: x[1])
    shortest = vectors_and_norms[0]
    vec_x = [f"{c//x}x + {c%x}" if abs(c) > 10 else f"{c}" for c in shortest[0]]
    print(f"Norm: {shortest[1]}")
    print(f"Shortest Vector {vec_x}")
    print("\n")
    return shortest[0]

def get_optimal_lambda_bls(q, r, x):
    L = generate_lattice_matrix(q, r, 12)
    shortest = list(find_vectors_and_sort_by_norm(L, x))
    res = shortest[0]
    for i, coeff in enumerate(shortest[1:]):
        res = res + coeff*q^(i+1)
    return res

In [75]:
# BLS12-381 Constants: prime field (q), order (r), seed (x), final exp exponent (h), and optimal lambda (lam) 
x = -15132376222941642752 
r = (x**4-x**2+1)
q = ((x-1)**2)//3 * r + x
h = (q**12 - 1) // r 
lam = get_optimal_lambda_bls(q, r, x)
assert lam == -x + q, f"{lam} != {-x + q}"

m = lam // r
d = math.gcd(m, h)

Norm: sqrt(228988810152649578064853576960394133505)
Shortest Vector ['-1x + 0', '1', '0', '0']




In [76]:
# Extension field definition.  
Fq = GF(q, proof=False)
# Define the polynomial ring over Fq
R.<w> = PolynomialRing(Fq)
# Irreducible polynomial
p_irr = w^12 - 2*w^6 + 2
# Define the extension field Fq12
F12 = Fq.extension(p_irr, 'z')
# Now you can perform arithmetic in Fq12 using z as the variable. 
z = F12.gen()

# unit test for arithmetic. 
A=0x20dde72a1f26fb32117c38b9ab89c2e0e638aba1290173d65bfa8ba221ed3f90190726852411d1037547231c091fef2*z^11 + 0x1ea226756e915a0f8cb41c763165199e926c0751205bf09de892a14e4db23f24f8e0f69e63e44a809f9a4a4fb291c86*z^10 + 0x5ecc96dd05013c8df4d8ffa527dae6f173788c01a2fc5ffe3e569718b7524dc5d17dbdf0f4bf086463b45db7c560180*z^9 + 0x16d34ae53b311307612baa56e4a3ea8d077e26c8ed051454903da860abafc400cf9e9ac14f4d48d1e9885c310bfb43d2*z^8 + 0xb9a7ea4ebca194d594fa8b3a72eac45f1ca7ed63e3080293d15c881034dcdc6eafbb661410f19d89a2e4534b087903b*z^7 + 0x80aec6ae8f12370041aad4560c8371d8c56eb85772b4ba42f9c234dbb9909696861b9f7cd33e4bbf4604e11acf35fa4*z^6 + 0x53c172223ca70a518432810b336fa09ef3ae60c50592a86fc8d7abcc87bf21f689383e59b7abc7aa9059c94b2e4a7d9*z^5 + 0xd6926dda88fa4a9b50f0b16152c7691b417cc3dd6126135b8111953c97083e0ea15383b4b6937c22b76aea97135194d*z^4 + 0x9c0bba2374da4614b56eb52d46a095f39adf6f4d17fb3adc4f11358a9a3f57af606979272fd40832a0a09ba96905bf1*z^3 + 0x10e79c4372cebba21f8ae83c7ca563c51fd7d770ba49ea1cb1bf300ab7adb2be7f8f20b11fa2b26680263ddd223f7583*z^2 + 0x3411f6efc5c89cd68eb581537a4bdc1ed701c3dc225a8712adbf3dcca7a8cc18f6dcf54f6972c4d76ac0da799e85663*z^1 + 0xde75be859de164a84663e69d73703d6a47034c039003775266c1178f02407ca4e3ddcb453630cbd09ab4db33c61ea5a*z^0
A_SQ=0xd9ab8c69f80e5386963046e8f1e3b4d8217326a0d0e0043e6dcf662e7d8726398047b1f166aa8aa6f7f0970f3785554*z^11 + 0x129ed9d2e52fccde336a817097ed6651f7b5083b71cffa137720044a22f6e58df0bca5ab073b3382ecf27acd3001fb6d*z^10 + 0x19de179279f040bd63ad84a4100559b176dbb9d79464267bcdfbb8729211b85aec687b54cbeb0d8877b78e8e15ddf04b*z^9 + 0x13403d09bf9c02f7035d970cbc4a05c5e1e2df7876a944a34f27093c57543277cd114a2665811ac1a85bbd218112b0da*z^8 + 0x58629c6eb6f7ca43218d7daabd194058b1a8f550c05c98809780834731aa7a6fdc231e48abdd93275ed7f37679886aa*z^7 + 0xcc416c009a5fddb29f513aceb02687d5d99155b627623fdb5dfd3ad2ede5b76f17add850b144aaa0be227008f5c8579*z^6 + 0xec0d603c11d283b05de0fed772d8bbaa9cf9b386e58f8a603639e0eb38cadbbdb684d1b1fdbd8d4592f2d73062df457*z^5 + 0x24a4836a6c278447c3bacf01d3b9f1189bc890266fad7c681fcfd708a049212532f21796114f65efbafa26e8eed51b4*z^4 + 0x104816a7d8ae9092cf0e95fcb160f041988b33b36907c7a301b3e559a6550f83bb6b6e7ffaa865803b618642e38447e4*z^3 + 0xd35a6eb124ef554b125449c70bc9bdc47dcc131ee55da21e17f3be40f26e43901a25b973e4620d9df8d4cde1b922bc1*z^2 + 0x5bfa5c2e4f1106326b0c05eff1da3ed83971a2281800b2c7368ed36aca078e81541b4aaa32d0523122b156265184d66*z^1 + 0x16c9600e19c3f6bb14dac7456de1c660d0de421282dd2944aa51c13e2efd74e55322c4b3a6be5fa416adb8273e773b9d*z^0
assert A**2 == A*A == A_SQ


In [77]:
# Define F as a miller loop result such that F = 1 * c ^ R
F = 0x910a0487f9fa577fbc7e8829a5ed0375612963137788cfae7ce2470e1d56ed586d74aba6cc0dd10322cab24e0b6bcb9*z^10 + 0x0*z^1 + 0x14aee3b861d5bfebc5a46ccc58413608138585cf5093ea6cb7f36654753a953df2ea0502153305c43744636cc554423c*z^8 + 0x0*z^1 + 0x134e9dd199ec0803b12f4f1f3bfddcfdf72ff98768eead1582f6c9ea4f2a6936655c9253c5676135d67352d27129f0f9*z^6 + 0x0*z^1 + 0x2568fe1eed3d5ad3fdb63f10ec2f612f4490a476f5736a12b2b5e817bac6a3fa07d10a131d8d391218633378a3cb502*z^4 + 0x0*z^1 + 0x10cffdbc31164968f1450032215deb0a0fafc1c3f9d14de45958d43a2bcb4530cfd40f148d4187fd288c52af6f7d78b1*z^2 + 0x0*z^1 + 0x16072615f8e82e622b45c42f6dfaece25e4810446ee6550bcc2cf111c30277326f9a40d32bb3043fa2bf0cb2490da48a*z^0

assert F**h == 1

def get_rth_root(f):
    """
    Computes x such that x^r = f
    """
    r_inv = pow(r, -1, h)
    res = f**r_inv
    return res

# Apply Theorem 1. 
c = get_rth_root(F)
assert c**r == F 